In [16]:
import numpy as np
import pandas as pd

import os

import transformers
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder

from transformers import AutoTokenizer, BertModel, DistilBertModel
from transformers import AutoModel, BertForSequenceClassification, BertTokenizer

from datasets import Dataset, ClassLabel

import torch
from torch.utils.data import Dataset as TorchDataset, DataLoader
from torch.nn import TripletMarginLoss
from torch.optim import Adam
from tqdm import tqdm

import neptune.new as neptune

In [28]:
dataset_df = pd.read_csv('dataset/embeddings_smol.csv')

In [29]:
dataset_df

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,labels
0,0.286959,-0.072739,0.753478,0.029065,-0.044117,0.796469,-0.080700,-0.116337,0.106141,0.664438,...,-0.081861,-0.296287,0.481009,-0.632536,-0.181246,-0.031159,0.367673,-0.146828,-0.786743,9
1,0.357129,-0.174773,0.603911,0.026667,0.094043,0.766938,-0.142196,-0.122094,0.022330,0.673558,...,-0.077524,-0.412776,0.496480,-0.645392,-0.021252,0.059162,0.468731,0.179680,-0.866986,2
2,0.395993,-0.010106,0.656082,0.130611,-0.051991,0.719535,-0.271543,0.046218,0.007320,0.625046,...,-0.233596,-0.333096,0.483195,-0.732185,-0.191192,0.217195,0.477800,-0.053176,-0.807459,13
3,0.334255,-0.037806,0.703386,0.064638,0.027087,0.798698,-0.184298,-0.088888,0.138465,0.644953,...,-0.089683,-0.295783,0.493706,-0.596528,-0.298668,0.063497,0.299139,-0.043658,-0.803471,33
4,0.320405,0.077909,0.684145,0.079528,0.158576,0.772303,-0.010687,0.017744,0.207142,0.698566,...,-0.090975,-0.369075,0.639004,-0.637897,-0.158942,-0.108213,0.344758,-0.076982,-0.767144,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3620,0.421578,-0.019629,0.656779,0.174148,-0.021393,0.840218,-0.100070,-0.011857,0.167257,0.692656,...,-0.110019,-0.271366,0.487649,-0.640063,-0.209200,0.021151,0.340553,-0.006638,-0.781998,26
3621,0.368945,-0.013736,0.691660,0.126277,0.095280,0.758165,-0.146781,-0.064218,0.114663,0.645248,...,-0.107440,-0.471480,0.559715,-0.732553,-0.126775,-0.059502,0.456765,-0.030017,-0.840898,11
3622,0.265164,-0.116977,0.706452,0.182964,0.142396,0.793411,0.043281,-0.093050,0.318244,0.642137,...,-0.205144,-0.313186,0.530374,-0.650075,-0.190040,0.123436,0.518753,-0.017935,-0.817771,19
3623,0.329796,-0.057874,0.734024,0.115979,0.116618,0.739115,-0.073479,-0.093619,0.288657,0.674363,...,-0.155023,-0.314374,0.598662,-0.596325,-0.178441,-0.073083,0.377727,0.009060,-0.792576,0


In [40]:
class TweetDataset(TorchDataset):
    def __init__(self, dataset: pd.DataFrame):
        self.labels = torch.Tensor(dataset['labels'])
        self.embeddings = torch.Tensor(dataset.drop(['labels'], axis=1).to_numpy())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        anchor = self.embeddings[item]
        anchor_class = self.labels[item]

        positive_indices = self.labels == anchor_class
        positive_indices = positive_indices.nonzero()
        positive_idx = positive_indices[torch.randint(high=len(positive_indices), size=(1, ))[0]]
        #positive_example = self.input_ids[positive_idx].flatten()
        #positive_attention = self.attention_mask[positive_idx]
        positive_example = self.embeddings[positive_idx]

        negative_indices = self.labels != anchor_class
        negative_indices = negative_indices.nonzero()
        negative_idx = negative_indices[torch.randint(high=len(negative_indices), size=(1, ))[0]]
        #negative_example = self.input_ids[negative_idx].flatten()
        #negative_attention = self.attention_mask[negative_idx]
        negative_example = self.embeddings[negative_idx]

        return anchor, positive_example, negative_example

In [41]:
dataset = TweetDataset(dataset_df)

torch.Size([3625, 768])